<a href="https://colab.research.google.com/github/Gilgamesh60/OCR_project/blob/main/Subspace_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libraries

**Pytesseract for extracting texts from images**

In [ ]:
!pip3 install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt install tesseract-ocr

**Date detectors for finding dates from the text**

In [ ]:
!pip install datefinder

In [ ]:
!pip install dateparser

In [ ]:
!pip install date_detector

**Haar-cascades for face detection**

In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml

In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_eye.xml \

# Importing libraries



In [ ]:
import requests
import os
import pytesseract
import numpy as np
import pandas as pd
from PIL import Image
import datetime
import re
from dateparser import parse
import string
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from date_detector import Parser
import string
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
#Converting the JSON file into csv format
with open('data_load.json', encoding='utf-8') as inputfile:
    df1 = pd.read_json(inputfile)
df1.to_csv('file.csv', encoding='utf-8', index=False)

In [ ]:
#Loading the given dataset
df1=pd.read_csv('file.csv')
df1

,expiry_image,expiring_at,expiring_at_from_model,created_at,whatsub_plan
0,https://cdn.subspace.money/w_subscriptions/Xk5...,2023-05-13,2023-05-13,2022-05-15 04:27:17.955815+00:00,"{'duration': 3, 'duration_type': 'months'}"
1,https://cdn.subspace.money/w_subscriptions/PKV...,2023-03-29,NaN,2022-06-10 11:28:46.053421+00:00,"{'duration': 1, 'duration_type': 'years'}"
2,https://cdn.subspace.money/w_subscriptions/80E...,2022-12-27,2022-12-27,2022-07-21 16:30:53.316893+00:00,"{'duration': 1, 'duration_type': 'months'}"
3,https://cdn.subspace.money/w_subscriptions/JIX...,2022-10-22,2022-08-22,2022-07-22 15:02:53.685566+00:00,"{'duration': 1, 'duration_type': 'months'}"
4,https://cdn.subspace.money/w_subscriptions/p4r...,2022-07-12,NaN,2022-06-12 05:16:00.847964+00:00,"{'duration': 1, 'duration_type': 'months'}"
...,...,...,...,...,...
6723,https://cdn.subspace.money/w_subscriptions/OYl...,2023-10-07,2023-10-07,2022-10-07 16:03:36.596915+00:00,"{'duration': 12, 'duration_type': 'months'}"
6724,https://cdn.subspace.money/w_subscriptions/5av...,2022-11-07,NaN,2022-10-07 16:06:06.037587+00:00,"{'duration': 1, 'duration_type': 'months'}"
6725,https://cdn.subspace.money/w_subscriptions/X4e...,2022-12-31,NaN,2022-10-07 17:25:59.667282+00:00,"{'duration': 1, 'duration_type': 'months'}"
6726,https://cdn.subspace.money/w_subscriptions/qmT...,2022-11-07,NaN,2022-06-14 08:22:25.590519+00:00,"{'duration': 1, 'duration_type': 'months'}"


# Saving images in drive folder

In [ ]:
#reads image using given image url and saves it in the specified folder
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
counter=0
for i in df1["expiry_image"]:
  persist_image('/content/drive/MyDrive/subspace',i,counter)
  counter+=1

# Baseline Regex Model

In [ ]:
#finds all the dates of specified regex patterns
def find_date(text):
    regex = r"((19|20)?\d{1,2}\s?[-/]\s?\d{1,2}\s?[-/]\s?(19|20)?\d{2})|"\
    r"((Jan|Feb|Mar|Apr|May|Jun|June|Jul|Aug|Sept|Sep|Oct|Nov|Dec)"\
    r"\s?\d{1,2}\s?[,']?\s?(19|20)?\d{2})|(\d{1,2}\s?[-/]?\s?"\
    r"(Jan|Feb|Mar|Apr|May|Jun|June|Jul|Aug|Sept|Sep|Oct|Nov|Dec)"\
    r"\s?[',-/]?\s?(19|20)?\d{1,2})"
    pattern = re.compile(regex, flags=re.IGNORECASE)
    matches = list(re.finditer(pattern, text))
    if len(matches)==0:
        return None
    date = matches[0].group(0)
    return date

In [ ]:
#extracting dates from all images in the subspace folder
dates=[]
def extract_date(path):
  try:
    text = pytesseract.image_to_string(Image.open(path))
    date=find_date(text)
    if date is None:
      dates.append(np.nan)
    else:
      dates.append(date.strftime("%Y-%m-%d"))
  except:
    pass

In [ ]:
for i in range(len(df1.index)):
  extract_date(f'/content/drive/MyDrive/subspace/jpg_{i}.jpg')

In [ ]:
Dates={'dates':dates}
df2=pd.DataFrame(Dates)
df2.to_csv('result_dates.csv')

In [ ]:
df2=pd.read_csv('result_dates.csv')
df2.drop('Unnamed: 0',axis=1,inplace=True)
df2.rename(columns={'0':'Dates'},inplace=True)
df2

,Dates
0,2023-05-13
1,2022-03-29
2,NaN
3,2022-08-22
4,NaN
...,...
6722,2023-10-07
6723,NaN
6724,NaN
6725,NaN


In [ ]:
df=pd.concat([df1,df2],axis=1)
df

,expiry_image,expiring_at,expiring_at_from_model,created_at,whatsub_plan,Dates
0,https://cdn.subspace.money/w_subscriptions/Xk5...,2023-05-13,2023-05-13,2022-05-15 04:27:17.955815+00:00,"{'duration': 3, 'duration_type': 'months'}",2023-05-13
1,https://cdn.subspace.money/w_subscriptions/PKV...,2023-03-29,NaN,2022-06-10 11:28:46.053421+00:00,"{'duration': 1, 'duration_type': 'years'}",2022-03-29
2,https://cdn.subspace.money/w_subscriptions/80E...,2022-12-27,2022-12-27,2022-07-21 16:30:53.316893+00:00,"{'duration': 1, 'duration_type': 'months'}",NaN
3,https://cdn.subspace.money/w_subscriptions/JIX...,2022-10-22,2022-08-22,2022-07-22 15:02:53.685566+00:00,"{'duration': 1, 'duration_type': 'months'}",2022-08-22
4,https://cdn.subspace.money/w_subscriptions/p4r...,2022-07-12,NaN,2022-06-12 05:16:00.847964+00:00,"{'duration': 1, 'duration_type': 'months'}",NaN
...,...,...,...,...,...,...
6723,https://cdn.subspace.money/w_subscriptions/OYl...,2023-10-07,2023-10-07,2022-10-07 16:03:36.596915+00:00,"{'duration': 12, 'duration_type': 'months'}",NaN
6724,https://cdn.subspace.money/w_subscriptions/5av...,2022-11-07,NaN,2022-10-07 16:06:06.037587+00:00,"{'duration': 1, 'duration_type': 'months'}",NaN
6725,https://cdn.subspace.money/w_subscriptions/X4e...,2022-12-31,NaN,2022-10-07 17:25:59.667282+00:00,"{'duration': 1, 'duration_type': 'months'}",NaN
6726,https://cdn.subspace.money/w_subscriptions/qmT...,2022-11-07,NaN,2022-06-14 08:22:25.590519+00:00,"{'duration': 1, 'duration_type': 'months'}",2023-03-23


In [ ]:
No_of_null_values=df['Dates'].isnull().sum()
Total_dates=len(df.index)
Basic_Success_Rate=(1-(No_of_null_values/Total_dates))*100
Basic_Success_Rate

49.68787158145065

**As you can see this success rate is extremely low .So we will do some optimizations to increase our success rate**

# Image Processing

In [ ]:
def image_smoothening(img):
  # thresholding with less noise 
  ret1, th1 = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
  blur = cv2.GaussianBlur(th1, (5, 5), 0)
  ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY)
  return th2

def image_process(path):
  img=cv2.imread(path)
  gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  ret,thresh1 = cv2.threshold(gray,170, 255, cv2.THRESH_BINARY)
  kernel = np.ones((1, 1), np.uint8)
  thresh = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
  thresh = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
  smooth_img = image_smoothening(gray)
  final_img = cv2.bitwise_or(smooth_img, thresh)
  return final_img

# Useless Images Detection

In [ ]:
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
def face_detector(img):
   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces  
   faces = face_cascade.detectMultiScale(gray, 1.1, 4)
   eyes = eye_cascade.detectMultiScale(gray,1.1,4)
   if type(faces) is tuple or type(eyes) is tuple:
     return True
   else:
     return False

# Text Processing

In [ ]:
#Removing Punctuations from text strings
def text_process(test_str):
  test_str = test_str.translate(str.maketrans('', '', string.punctuation))
  return test_str

# Model Building

In [ ]:
!pip install date_detector
from date_detector import Parser
parser=Parser()

In [ ]:
data=[]
cnts=0
from PIL import Image
for i in range(len(df.index)):
  img=cv2.imread(f'/content/drive/MyDrive/subspace/jpg_{i}.jpg')
  if face_detector(img) is False:
    cnts+=1
    pass
  else:
    l=[]
    try:
      text=pytesseract.image_to_string(Image.open(f'/content/drive/MyDrive/subspace/jpg_{i}.jpg'))
      text=text_process(text)
      date=find_date(text)
      for match in parser.parse(text):
        l.append(match.date.strftime("%d/%m/%Y"))
      if len(l)==0:
        if date is not None:
          data.append(date)
        else:
          img=image_process(f'/content/drive/MyDrive/subspace/jpg_{i}.jpg')
          text=pytesseract.image_to_string(img)
          text=text_process(text)
          m=[]
          for match in parser.parse(text):
               m.append(match.date.strftime("%d/%m/%Y"))
          if len(m)!=0:
             data.append(m[0])
          else:
             data.append(None)
      else:
         data.append(l[0])
      print(f'The date from {i}th image is extracted')
    except:
       pass

In [ ]:
#Removing useless images from the dataset
for i in range(len(df1.index)):
   img=cv2.imread(f'/content/drive/MyDrive/subspace/jpg_{i}.jpg')
   if face_detector(img) is False:
     df1.drop(i,axis=0,inplace=True)

In [ ]:
df1.to_csv('New_dataset.csv')

# Conclusion

In [ ]:
Total_images=len(df.index)
No_of_useless_images=cnt
No_of_useful_images=len(df.index)-No_of_useless_images
print(f'The number of useless images is :{No_of_useless_images}')
print(f'The percentage of useless images is:{(No_of_useless_images/Total_images)*100}%')
print(f'The number of useful images is :{No_of_useful_images}')
print(f'The percentage of useful images is:{(No_of_useful_images/Total_images)*100}%')

The number of useless images is :705
The percentage of useless images is:10.47859690844233%
The number of useful images is :6023
The percentage of useful images is:89.52140309155767%


In [ ]:
Data={'Dates':data}
Df=pd.DataFrame(Data)
Df.to_csv('Final_dates.csv')

In [ ]:
Null_values=data.count(None)
All_dates=No_of_useful_images
Success_rate=(1-(Null_values/All_dates))*100
Success_rate

71.875

In [ ]:
Df

,Dates
0,29/03/2022
1,27/12/2022
2,22/08/2022
3,None
4,22/08/2022
...,...
6017,17/09/2022
6018,None
6019,07/10/2023
6020,None
